In [5]:
import xarray as xr
import numpy as np
import os
from datetime import datetime, timedelta

OUTPUT_LOCATION = "/Odyssey/public/glonet/"


In [41]:
in1 = xr.open_dataset("../../../woneydb/in1.nc")
in2 = xr.open_dataset("../../../woneydb/in2.nc")
in3 = xr.open_dataset("../../../woneydb/in3.nc")

out = xr.open_dataset("../../../woneydb/glonet_out/forecast_10days_from_2025-06-30.nc")
out = out.isel(time = [5, 6])


In [6]:
test = xr.open_dataset("../../../output_glonet/forecast_7days_from_2025-07-01.nc")
test["time"].dt.date.values[0] - timedelta(days=1)

datetime.date(2025, 7, 1)

In [42]:

out1 = out.sel(depth = slice(0, 30))
out2 = out.sel(depth = slice(40, 800)).drop_vars(["zos"])
out3 = out.sel(depth = slice(900, 5500)).drop_vars(["zos"])

In [48]:
(out1.to_array(dim = "variable")                              # Convert to xarray DatArray
                    .stack(ch=("variable", "depth"))                        # Merge two dimensions one dimension "ch"
                    .reset_index("ch", drop=True)                           # Clear void coordinates 
                    .assign_coords(ch=np.arange(len(out1.data_vars) * out1.sizes["depth"])+1) # define values of ch 
                    .transpose("time", "ch", "latitude", "longitude")       # Reshape in order
                    .rename({"latitude" : "lat", "longitude" : "lon"})      # Rename into input Dataset config
                    .to_dataset(name="data"))      


<xarray.Dataset> Size: 77MB
Dimensions:  (lat: 672, lon: 1440, time: 2, ch: 5)
Coordinates:
  * lat      (lat) float64 5kB -78.0 -77.75 -77.5 -77.25 ... 89.25 89.5 89.75
  * lon      (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time     (time) datetime64[ns] 16B 2025-07-06 2025-07-07
  * ch       (ch) int64 40B 1 2 3 4 5
Data variables:
    data     (time, ch, lat, lon) float64 77MB nan nan nan ... 28.54 28.45 29.56

In [49]:
in1

<xarray.Dataset> Size: 39MB
Dimensions:  (lon: 1440, lat: 672, ch: 5, time: 2)
Coordinates:
  * lon      (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * lat      (lat) float64 5kB -78.0 -77.75 -77.5 -77.25 ... 89.25 89.5 89.75
  * ch       (ch) int32 20B 0 1 2 3 4
  * time     (time) datetime64[ns] 16B 2025-06-30 2025-07-01
Data variables:
    data     (time, ch, lat, lon) float32 39MB ...

In [46]:
test = xr.open_dataset("test.nc")
test

<xarray.Dataset> Size: 77MB
Dimensions:                        (lat: 672, lon: 1440, time: 2, ch: 5)
Coordinates:
  * lat                            (lat) float64 5kB -78.0 -77.75 ... 89.5 89.75
  * lon                            (lon) float64 12kB -180.0 -179.8 ... 179.8
  * time                           (time) datetime64[ns] 16B 2025-07-06 2025-...
  * ch                             (ch) int64 40B 1 2 3 4 5
Data variables:
    __xarray_dataarray_variable__  (time, ch, lat, lon) float64 77MB ...

In [ ]:
import xarray as xr
import numpy as np

# Reshape Forecast Dataset to GLONET input (3 files) Dataset shape. 
# i.e. flatten depth dimension and passe it to channel with variables.


def reshapeDataset(in_ds : xr.Dataset) -> xr.Dataset:
        
    out_ds = (in_ds.to_array(dim = "variable")                              # Convert to xarray DatArray
                        .stack(ch=("variable", "depth"))                        # Merge two dimensions one dimension "ch"
                        .reset_index("ch", drop=True)                           # Clear void coordinates 
                        .assign_coords(ch=np.arange(len(in_ds.variables) * in_ds.sizes["depth"])) # define values of ch 
                        .transpose("time", "ch", "latitude", "longitude")       # Reshape in order
                        .rename({"latitude" : "lat", "longitude" : "lon"})      # Rename into input Dataset config
                        .to_dataset)                                            # Reconvert to Dataset
        
    return out_ds

# Divide forcast NetCDF file in to 3 files by states depth
def divideIntoThree(input_nc : str, 
                    cycle : int) :
    
    # Extract date for output name
    forecast_date = input_nc["time"].values[cycle - 1]
    
    # Read netCDF file and pick only two last timeset
    last_two_day_forecast = xr.open_dataset(input_nc).isel(time = [cycle-2, cycle-1])
    
    # Slice datatset by depth first 
    surface = last_two_day_forecast.isel(depth = 0, missing_dims="ignore")
    deep = last_two_day_forecast.sel(depth = slice(40, 800))
    deeper = last_two_day_forecast.sel(depth = slice(900, 5500))
    
    # Make a dictionary for iterattion
    ds_map = {
            "1" : surface,
            "2" : deep,
            "3" : deeper
    }
    
    # Write input file
    out_dir = OUTPUT_LOCATION + "/init??"
    os.makedirs(out_dir, exist_ok=True)
    
    in123 = []
    
    for i in ds_map :
        reshapeDataset(i).to_netcdf(f"{out_dir}/{forecast_date}_autoregress_init_states")
        in123.append(reshapeDataset(i))
        
    return in123
